# Solving MNIST with ANN

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

/home/igor/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Data preparation
- importing data
- scaling data
- train-test splitting

In [2]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
mnist=fetch_mldata('MNIST original')
stdsc=StandardScaler()
#X,y=pd.DataFrame(mnist['data']), pd.DataFrame(mnist['target'])
X_all,y_all=mnist['data'], mnist['target'].astype(int)
X_all=stdsc.fit_transform(X_all)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.20, random_state=42)

/home/igor/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


- function for selcting batches:

In [3]:
def select_batch(X,y,batchsize):
    random_indices=np.random.randint(X.shape[0],size=batchsize)
    return X[random_indices], y[random_indices]

### Defining the neural network for classification


In [4]:
tf.reset_default_graph()
n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10
X=tf.placeholder(dtype=tf.float32,shape=(None, n_inputs), name="X")
y=tf.placeholder(shape=(None), dtype=tf.int32, name="y")

with tf.name_scope("DNN"):
    hidden1=tf.layers.dense(inputs=X, units=n_hidden1, activation=tf.nn.relu, name='hidden1')
    hidden2=tf.layers.dense(inputs=hidden1, units=n_hidden2, activation=tf.nn.relu, name='hidden2')
    logits=tf.layers.dense(inputs=hidden2, units=n_outputs, activation=None, name='logits')
    
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(xentropy, name="loss")
    
learning_rate=0.05
with tf.name_scope("train"):
    #optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    optimizer=tf.train.AdagradOptimizer(learning_rate=learning_rate)
    training_op=optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(targets=y,predictions=logits,k=1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))
    
init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [5]:
from datetime import datetime
now=datetime.utcnow().strftime("%Y%m%d_%H%M%S")
root_logdir="dnn_mnist_logs"
logdir="{}/run-{}/".format(root_logdir, now)
# logdir2=root_logdir+'/run-'+str(now)+'/'

# node for evaluating loss and outputing it in tensorboard-readable string:
loss_summary=tf.summary.scalar('cross-entropy loss',loss)
# object that can write summaries into logfile
file_writer=tf.summary.FileWriter(logdir, tf.get_default_graph())

INFO:tensorflow:Summary name cross-entropy loss is illegal; using cross-entropy_loss instead.


### Training the neural network:

In [6]:
n_epochs=20
batch_size=50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0]//batch_size):
            X_batch, y_batch=select_batch(X_train,y_train,batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accurracy_train=accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        accurracy_test=accuracy.eval(feed_dict={X: X_test, y: y_test})
        print('Epoch ',epoch,'-- train accuracy: ', accurracy_train, ', test accuracy: ', accurracy_test)
        
        loss_summary_str=loss_summary.eval(feed_dict={X:X_test, y:y_test})
        file_writer.add_summary(loss_summary_str, epoch)
        
    saver.save(sess, "./saved_models/dnn_mnist_model.ckpt")
# important to close in order to be able to directly see the results in the tensorboard:
file_writer.close() 

Epoch  0 -- train accuracy:  0.96 , test accuracy:  0.95857143
Epoch  1 -- train accuracy:  1.0 , test accuracy:  0.968
Epoch  2 -- train accuracy:  1.0 , test accuracy:  0.9705
Epoch  3 -- train accuracy:  1.0 , test accuracy:  0.97235715
Epoch  4 -- train accuracy:  1.0 , test accuracy:  0.9742857
Epoch  5 -- train accuracy:  1.0 , test accuracy:  0.9735714
Epoch  6 -- train accuracy:  1.0 , test accuracy:  0.9762143
Epoch  7 -- train accuracy:  1.0 , test accuracy:  0.97664285
Epoch  8 -- train accuracy:  1.0 , test accuracy:  0.9762857
Epoch  9 -- train accuracy:  1.0 , test accuracy:  0.9765
Epoch  10 -- train accuracy:  1.0 , test accuracy:  0.9762857
Epoch  11 -- train accuracy:  1.0 , test accuracy:  0.97642857
Epoch  12 -- train accuracy:  1.0 , test accuracy:  0.9765
Epoch  13 -- train accuracy:  1.0 , test accuracy:  0.97642857
Epoch  14 -- train accuracy:  1.0 , test accuracy:  0.9772143
Epoch  15 -- train accuracy:  1.0 , test accuracy:  0.9769286
Epoch  16 -- train accura